In [0]:
%run "/Workspace/Repos/ashwin.nair8089@gmail.com/inshorts/library/connector"

In [0]:
%run "/Workspace/Repos/ashwin.nair8089@gmail.com/inshorts/library/password_manager"

In [0]:
conn, success, message = snowflake_connector("ashwin009", get_password('snf_conn'), "WSYCJNH-JF51761",'COMPUTE_WH','INSHORTS','ETL')

In [0]:
cursor = conn.cursor()

In [0]:

cursor.execute('Truncate table foundation.user_churn')
print(cursor.fetchall())
cursor.execute(f'''INSERT INTO FOUNDATION.user_churn
WITH category_transitions AS (
    SELECT 
        deviceid, 
        DATE(eventtimestamp) AS event_date,
        LAG(eventname) OVER (PARTITION BY deviceid ORDER BY eventtimestamp) AS previous_eventname,
        eventname AS current_eventname
    FROM ETL.event
)
SELECT 
    event_date,
    previous_eventname,
    current_eventname,
    COUNT(DISTINCT deviceid) AS churned_users,
    COUNT(DISTINCT deviceid) / SUM(COUNT(DISTINCT deviceid)) OVER (PARTITION BY previous_eventname) AS churn_rate
FROM category_transitions
WHERE previous_eventname IS NOT NULL
GROUP BY event_date, previous_eventname, current_eventname;''')
print(cursor.fetchall())